In [171]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder


date_cols = ['Stime', 'Ltime']
fields1 = ['ct_flw_http_mthd', 'is_ftp_login', 'ct_ftp_cmd', 'Stime', 'Ltime']
fields2 = ['is_ftp_login', 'ct_ftp_cmd', 'Stime', 'Ltime']
fields3 = ['ct_ftp_cmd', 'Stime', 'Ltime']
cols = [
    'srcip',
    'sport',
    'dstip',
    'dsport',
    'proto',
    'state',
    'dur',
    'sbytes',
    'dbytes',
    'sttl',
    'dttl',
    'sloss',
    'dloss',
    'service',
    'Sload',
    'Dload',
    'Spkts',
    'Dpkts',
    'swin',
    'dwin',
    'stcpb',
    'dtcpb',
    'smeansz',
    'dmeansz',
    
    'trans_depth',
    'res_bdy_len',
    'Sjit',
    'Djit',
    'Stime',
    'Ltime',
    'Sintpkt',
    'Dintpkt',
    'tcprtt',
    'synack',
    'ackdat',
    'is_sm_ips_ports',
    'ct_state_ttl',
    
#     'ct_flw_http_mthd',
#     'is_ftp_login',
#     'ct_ftp_cmd',
    
    'ct_srv_src',
    'ct_srv_dst',
    
#     'ct_dst_ltm',
#     'ct_src_ltm',
    
    'ct_src_dport_ltm',
    'ct_dst_sport_ltm',
    'ct_dst_src_ltm',
    'attack_cat',
    'Label',
]
fields5 = ['attack_cat', 'Stime', 'Ltime']


def strip_int(text):
    return int(text.strip(' '))

def ip_to_int(ip):
    z = ''
    parts = ip.split('.')
    for prt in range(3):
        if len(parts[prt]) == 3:
            z = parts[prt] + z
        elif len(parts[prt]) == 2:
            z = '0' + parts[prt] + z
        else:
            z = '00' + parts[prt] + z
    return int(z)

def encode_attack_cat(cat):
    mapping = {
        'Generic': 1,
        'Fuzzers': 2,
        'Exploits': 3,
        'DoS': 4,
        'Reconnaissance': 5,
        'Backdoor': 6,
        'Backdoors': 6,
        'Analysis': 7,
        'Shellcode': 8,
        'Worms': 9,
    }
    if (not cat):
        return 0
    return mapping[cat.strip(' ')]

def decode_attack_cat(encoded_cat):
    mapping = {
        0: '',
        1: 'Generic',
        2: 'Fuzzers',
        3: 'Exploits',
        4: 'DoS',
        5: 'Reconnaissance',
        6: 'Backdoors',
        7: 'Analysis',
        8: 'Shellcode',
        9: 'Worms',
    }
    return mapping[encoded_cat]

def encode_state(state):
    mapping = {
        'CON': 0,
        'FIN': 1,
        'INT': 2,
        'REQ': 3,
        'URH': 4,
        'RST': 5,
        'ECR': 6,
        'ECO': 7,
        'CLO': 8,
        'PAR': 9,
        'ACC': 10,
        'URN': 11,
        'MAS': 12,
    }
    return mapping[state]

def sanitize_port(port):
    return int(port.replace('-', 0))

converters = {
    'srcip': ip_to_int,
    'dstip': ip_to_int,
}

# Read csv
df = pd.read_csv(
    '../data/UNSW-NB15-BALANCED-TRAIN.csv',
    parse_dates=date_cols,
    usecols=cols,
    skipinitialspace=True,
    converters=converters
)

df['sport'] = df['sport'].replace('-', 0)
df['dsport'] = df['dsport'].replace('-', 0)
df['sport'] = df['sport'].apply(lambda x: int(str(x), 16))
df['dsport'] = df['dsport'].apply(lambda x: int(str(x), 16))
df['service'] = df['service'].replace('-', '')


le = LabelEncoder()
df['proto'] = le.fit_transform(df['proto'])
df['state'] = le.fit_transform(df['state'])
df['service'] = le.fit_transform(df['service'])
df['attack_cat'] = le.fit_transform(df['attack_cat'])

df.head()

/var/folders/r1/72xs0cc52vsfvf5nkpbx2fhh0000gn/T/ipykernel_63311/2613928297.py:146: DtypeWarning: Columns (1,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,ackdat,is_sm_ips_ports,ct_state_ttl,ct_srv_src,ct_srv_dst,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,Label
0,166059,349328,126171149,83,119,2,0.001050,146,178,31,...,0.000000,0,0,3,4,1,1,1,13,0
1,166059,157526,126171149,33,113,5,4.264797,2934,3738,31,...,0.000134,0,0,5,5,5,5,8,13,0
2,176045175,4163,126171149,83,119,6,0.000003,114,0,254,...,0.000000,0,2,17,17,14,14,17,7,1
3,176045175,4163,126171149,83,119,6,0.000003,114,0,254,...,0.000000,0,2,25,25,25,25,25,7,1
4,166059,214130,126171149,66601,113,5,0.052957,3718,43086,31,...,0.000130,0,0,5,8,1,1,2,13,0


In [162]:
df.describe()

,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,ackdat,is_sm_ips_ports,ct_state_ttl,ct_srv_src,ct_srv_dst,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,Label
count,4.497960e+05,449796.000000,4.497960e+05,4.497960e+05,449796.000000,449796.000000,449796.000000,4.497960e+05,4.497960e+05,449796.000000,...,449796.000000,449796.000000,449796.000000,449796.000000,449796.000000,449796.000000,449796.000000,449796.000000,449796.000000,449796.000000
mean,9.646951e+07,166478.437687,1.284399e+08,4.295583e+04,114.453261,5.085528,0.694259,5.092681e+03,2.277625e+04,138.566168,...,0.006775,0.000972,0.931353,13.858387,13.683721,8.416918,6.496405,12.401215,9.646331,0.500000
std,8.570929e+07,139797.213314,1.231686e+07,1.140547e+06,12.583730,1.293619,13.697272,1.157077e+05,1.449404e+05,109.317462,...,0.026773,0.031155,0.982520,13.818018,13.880075,11.033970,7.860205,14.575122,3.540252,0.500001
min,1.660590e+05,0.000000,2.240000e+02,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,...,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,1.660590e+05,4163.000000,1.261711e+08,8.300000e+01,113.000000,5.000000,0.000008,1.140000e+02,0.000000e+00,31.000000,...,0.000000,0.000000,0.000000,3.000000,3.000000,1.000000,1.000000,1.000000,7.000000,0.000000
50%,1.760452e+08,161571.000000,1.261711e+08,8.300000e+01,119.000000,5.000000,0.001058,2.640000e+02,1.780000e+02,62.000000,...,0.000000,0.000000,1.000000,7.000000,7.000000,2.000000,1.000000,4.000000,12.500000,0.500000
75%,1.760452e+08,291897.000000,1.261711e+08,5.923000e+03,119.000000,6.000000,0.098976,2.054000e+03,3.276000e+03,254.000000,...,0.000136,0.000000,2.000000,24.000000,24.000000,16.000000,13.000000,24.000000,13.000000,1.000000
max,2.411682e+08,415029.000000,2.411682e+08,5.389893e+08,132.000000,12.000000,8760.777344,1.435577e+07,1.465753e+07,255.000000,...,3.551121,1.000000,6.000000,67.000000,67.000000,67.000000,60.000000,67.000000,13.000000,1.000000


In [163]:
df.values

array([[166059, 349328, 126171149, ..., 1, 13, 0],
       [166059, 157526, 126171149, ..., 8, 13, 0],
       [176045175, 4163, 126171149, ..., 17, 7, 1],
       ...,
       [166059, 201106, 126171149, ..., 1, 13, 0],
       [176045175, 4163, 126171149, ..., 41, 7, 1],
       [166059, 354344, 126171149, ..., 1, 13, 0]], dtype=object)

In [218]:
# Correlation / Co-variance
threshold = 0.95

# Absolute value correlation matrix
corr_matrix = X_train.corr().abs()
corr_matrix.head()

/var/folders/r1/72xs0cc52vsfvf5nkpbx2fhh0000gn/T/ipykernel_63311/1953979357.py:5: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr_matrix = X_train.corr().abs()


,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,tcprtt,synack,ackdat,is_sm_ips_ports,ct_state_ttl,ct_srv_src,ct_srv_dst,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm
srcip,1.000000,0.222034,0.069884,0.026011,0.016012,0.629746,0.027422,0.004483,0.143386,0.906766,...,0.226278,0.199044,0.229962,0.028536,0.880653,0.531331,0.532702,0.538267,0.581248,0.595763
sport,0.222034,1.000000,0.020551,0.005123,0.142196,0.186677,0.039229,0.008771,0.037608,0.221860,...,0.091341,0.083550,0.089232,0.036666,0.238567,0.147156,0.149670,0.163015,0.205147,0.164443
dstip,0.069884,0.020551,1.000000,0.006534,0.075958,0.116439,0.091786,0.009526,0.028709,0.124582,...,0.046315,0.041724,0.045963,0.118612,0.165665,0.239410,0.239318,0.236287,0.250846,0.243760
dsport,0.026011,0.005123,0.006534,1.000000,0.020170,0.006027,0.003660,0.000549,0.000934,0.023362,...,0.004117,0.003585,0.004225,0.000958,0.021802,0.015872,0.016526,0.020258,0.021071,0.021173
proto,0.016012,0.142196,0.075958,0.020170,1.000000,0.073926,0.210411,0.006479,0.017822,0.016613,...,0.029046,0.026169,0.028823,0.265169,0.021316,0.231091,0.231724,0.212455,0.216525,0.219977


In [219]:
# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
upper.head()

,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,tcprtt,synack,ackdat,is_sm_ips_ports,ct_state_ttl,ct_srv_src,ct_srv_dst,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm
srcip,NaN,0.222034,0.069884,0.026011,0.016012,0.629746,0.027422,0.004483,0.143386,0.906766,...,0.226278,0.199044,0.229962,0.028536,0.880653,0.531331,0.532702,0.538267,0.581248,0.595763
sport,NaN,NaN,0.020551,0.005123,0.142196,0.186677,0.039229,0.008771,0.037608,0.221860,...,0.091341,0.083550,0.089232,0.036666,0.238567,0.147156,0.149670,0.163015,0.205147,0.164443
dstip,NaN,NaN,NaN,0.006534,0.075958,0.116439,0.091786,0.009526,0.028709,0.124582,...,0.046315,0.041724,0.045963,0.118612,0.165665,0.239410,0.239318,0.236287,0.250846,0.243760
dsport,NaN,NaN,NaN,NaN,0.020170,0.006027,0.003660,0.000549,0.000934,0.023362,...,0.004117,0.003585,0.004225,0.000958,0.021802,0.015872,0.016526,0.020258,0.021071,0.021173
proto,NaN,NaN,NaN,NaN,NaN,0.073926,0.210411,0.006479,0.017822,0.016613,...,0.029046,0.026169,0.028823,0.265169,0.021316,0.231091,0.231724,0.212455,0.216525,0.219977


In [220]:
# Drop duplicate columns by finding their correlations between each other
to_drop_raw = []

# Column for column in upper.columns if any upper[column] > threshold
for column in upper.columns:
    for index, correlation in enumerate(upper[column]):
        if (correlation > threshold):
            to_drop_raw.append(upper.columns[index])
            
to_drop = list(set(to_drop_raw))
to_drop

['dbytes', 'swin', 'ct_srv_src', 'tcprtt', 'dloss', 'ct_srv_dst', 'sbytes']

In [229]:
# Drop label columns and duplicate columns
label_cols = ['attack_cat', 'Label']
drop_cols = label_cols + to_drop
drop_cols
X = df.drop(columns=['attack_cat', 'Label'])
y = df['Label']

X.columns

Index(['srcip', 'sport', 'dstip', 'dsport', 'proto', 'state', 'dur', 'sbytes',
       'dbytes', 'sttl', 'dttl', 'sloss', 'dloss', 'service', 'Sload', 'Dload',
       'Spkts', 'Dpkts', 'swin', 'dwin', 'stcpb', 'dtcpb', 'smeansz',
       'dmeansz', 'trans_depth', 'res_bdy_len', 'Sjit', 'Djit', 'Stime',
       'Ltime', 'Sintpkt', 'Dintpkt', 'tcprtt', 'synack', 'ackdat',
       'is_sm_ips_ports', 'ct_state_ttl', 'ct_srv_src', 'ct_srv_dst',
       'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm'],
      dtype='object')

In [230]:
# X_train_validation includes training set and validation set
# X_test includes test set
# y_train_validation includes the labels for both the training set and validation set
# y_test includes the labels for the test set
X_train_validation, X_test, y_train_validation, y_test = train_test_split(X, y, test_size=0.2)

X_train_validation.head()

,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,tcprtt,synack,ackdat,is_sm_ips_ports,ct_state_ttl,ct_srv_src,ct_srv_dst,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm
18282,176045175,0,126171149,0,18,6,0.000003,200,0,254,...,0.000000,0.000000,0.000000,0,2,4,4,2,2,4
104009,166059,139809,126171149,83,119,2,0.001070,146,178,31,...,0.000000,0.000000,0.000000,0,0,5,3,2,1,2
270916,176045175,4163,126171149,83,119,6,0.000009,114,0,254,...,0.000000,0.000000,0.000000,0,2,35,35,22,22,35
240680,166059,272696,126171149,34,113,5,2.275536,12648,13410,31,...,0.001233,0.001120,0.000113,0,0,1,2,1,1,4
111902,166059,30056,126171149,128,113,5,1.004723,1580,10168,31,...,0.000652,0.000504,0.000148,0,0,2,4,1,1,1


In [231]:
# X_train is the training set
# X_validation is the validation set
# y_train is the labels for the training set
# y_validation is the labels for the validation set
X_train, X_validation, y_train, y_validation = train_test_split(X_train_validation, y_train_validation, test_size=0.2)

X_train.head()

,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,tcprtt,synack,ackdat,is_sm_ips_ports,ct_state_ttl,ct_srv_src,ct_srv_dst,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm
253630,126171149,4163,176045175,83,119,6,0.000008,264,0,60,...,0.000000,0.000000,0.00000,0,0,15,15,4,3,15
67511,166059,235106,126171149,37,113,5,0.089825,37322,3380,31,...,0.005086,0.004916,0.00017,0,0,1,1,1,1,1
413748,166059,333190,126171149,26753,113,5,0.015433,1540,1644,31,...,0.000562,0.000432,0.00013,0,0,11,10,4,1,5
442183,176045175,0,126171149,0,120,6,0.000003,200,0,254,...,0.000000,0.000000,0.00000,0,2,3,3,2,2,4
352584,126171149,291897,176045175,83,119,6,0.000007,264,0,60,...,0.000000,0.000000,0.00000,0,0,43,43,19,19,43


In [232]:
# Train
# model = DecisionTreeClassifier()
# model.fit(X_train, y_train)

# KNN - K-Nearest Neighbours Classifier




DecisionTreeClassifier()

In [233]:
# Validate model
predictions = model.predict(X_validation)

score = accuracy_score(y_validation, predictions)
score

0.9906625166740773